In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy import types
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

import os
import sys
sys.path
sys.path.append('../')
from connections import password, client_id, client_secret

In [2]:
# Client credentials
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id, 
    client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Establish SQL connection
connection_string = (f"root:{password}@localhost/spot_db")
engine = create_engine(f"mysql://{connection_string}")# , pool_recycle=3600, pool_pre_ping=True)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Save reference to the table
songs = Base.classes.songs
try:
    features = Base.classes.features
except:
    pass
# Create connection object
session = Session(engine)

In [4]:
# Grab song IDs
song_id = session.query(songs.ID).all()
id_list = list(np.ravel(song_id))

In [5]:
# Divide list of Ids into chunks of 100
def chunk_lists(list, n = 100): 
    for i in range(0, len(list), n):  
        yield list[i:i + n] 
# Must wrap output in list()
id_chunks = list(chunk_lists(id_list))

In [6]:
audio_features = []
for ids in id_chunks:
    audio_feature = sp.audio_features(tracks = ids)
    
    audio_features.append(audio_feature)

# Remove sublists
audio_features = [item for sublist in audio_features for item in sublist]

In [7]:
# Delete irrelevant categories
unrel_keys = ['type', 'uri', 'analysis_url', 'track_href']
for dict in audio_features:
    for key in unrel_keys:
        if key in dict:
            del dict[key]

In [8]:
features_df = pd.DataFrame(audio_features)
features_df.head()

,acousticness,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.9750,0.207,151920,0.0889,1b9WprGh5qohgtM3zb9aiO,0.105000,5,0.1130,-14.605,1,0.0337,67.390,3,0.0959
1,0.0168,0.757,281493,0.6800,1Th5b8AsauED9F7pGv8Yxi,0.015500,8,0.2750,-8.988,0,0.0410,122.772,4,0.2490
2,0.1530,0.673,390867,0.8130,1D6nV9TPfMnWm7UdVsDVfI,0.000029,5,0.0547,-10.966,1,0.0445,91.786,4,0.8960
3,0.0451,0.734,265693,0.7190,4dhYfCbHNHubV2wQduXRqO,0.007740,11,0.1030,-8.412,0,0.0325,109.994,4,0.8400
4,0.2880,0.552,323787,0.4330,6F7CjVIrwk7PddQUsoPmoo,0.000000,0,0.0859,-6.805,1,0.0520,142.563,4,0.2410


In [9]:
features_df.rename(columns = {'acousticness':'Acousticness', 'danceability':'Danceability', 
                              'duration_ms':'Duration_ms', 'energy':'Energy', 
                              'id':'ID', 'instrumentalness':'Instrumentalness', 
                              'key':'Key', 'liveness':'Liveness', 
                              'loudness':'Loudness', 'mode':'Mode', 
                              'speechiness':'Speechiness', 'tempo':'Tempo', 
                              'time_signature':'Time_Signature', 'valence':'Valence'}, inplace = True)

# Export to SQL

In [10]:
# Audio Features
engine.execute("USE spot_db")
features_df.to_sql(
    name = 'features', con = engine,
    if_exists = 'replace',
    chunksize = 75,
    dtype={'ID': types.VARCHAR(255)})
with engine.connect() as con:
    con.execute('ALTER TABLE `features` ADD PRIMARY KEY (`ID`);')